In [1]:
import numpy as np
import pandas as pd

import calendar
import datetime as dt
from collections import Counter

In [2]:
deliveries = pd.read_csv(r'Einkaufslieferzeile.csv', on_bad_lines='skip', sep=';', low_memory=False,encoding='utf-8-sig')
deliveries.head()

,Attached to Line No_,Auto Charge Doc_ Line No_,Auto Charge Doc_ No_,Billing Entry No_,Billing Reference No_,Blanket Order Line No_,Blanket Order No_,Budgeted FA No_,Buy-from Vendor No_,Contract Line No_,...,Use Duplication List,Use Tax,VAT Base Amount,VAT Bus_ Posting Group,VAT Calculation Type,VAT Prod_ Posting Group,VAT _,VDP Code,Variant Code,timestamp
0,0,0,NaN,0,NaN,0,NaN,NaN,K00421,0,...,0,0,"0,00000000000000000000",IL,0,MWST19,"19,00000000000000000000",NaN,NaN,000000005295C63A
1,0,0,NaN,0,NaN,0,NaN,NaN,K02028,0,...,0,0,"0,00000000000000000000",IL,0,MWST19,"19,00000000000000000000",NaN,NaN,0000000052EC85AD
2,0,0,NaN,0,NaN,0,NaN,NaN,K04610,0,...,0,0,"0,00000000000000000000",EU,1,MWST19,"0,00000000000000000000",NaN,NaN,0000000052B4AE3C
3,0,0,NaN,0,NaN,0,NaN,NaN,K04607,0,...,0,0,"0,00000000000000000000",IL,0,MWST19,"19,00000000000000000000",NaN,NaN,0000000052B4D12C
4,0,0,NaN,0,NaN,10000,RB19/0025,NaN,K04137,0,...,0,0,"0,00000000000000000000",IL,0,MWST19,"19,00000000000000000000",NaN,NaN,0000000052B4B8F5


In [3]:
deliveries.shape

(6012, 166)

In [4]:
# drop columns where all NaN values
deliveries=deliveries.dropna(axis=1, how='all')
deliveries.shape

(6012, 129)

In [6]:
# drop columns where all 0 values
deliveries=deliveries.loc[:, (deliveries != 0).any(axis=0)]
deliveries.shape

(6012, 90)

In [5]:
# drop columns where over 50% of values are null values
deliveries=deliveries.loc[:, deliveries.isnull().mean()<0.5]
deliveries.shape

(6012, 116)

In [7]:
# get list with column names
list(deliveries.columns)

['Blanket Order Line No_',
 'Buy-from Vendor No_',
 'Cross-Reference Type No_',
 'Demand Query Line No_',
 'Document No_',
 'Item Rcpt_ Entry No_',
 'Line No_',
 'No_',
 'Order Line No_',
 'Order No_',
 'Pay-to Vendor No_',
 'Pos_ No_',
 'Prod_ Order Line No_',
 'Routing Reference No_',
 'Allow Invoice Disc_',
 'Balance',
 'Bin Code',
 'Buy-from Address',
 'Buy-from City',
 'Buy-from Contact',
 'Buy-from Post Code',
 'Buy-from Vendor Name 2',
 'Buy-from Vendor Name',
 'Contract Amount',
 'Correction',
 'Cross-Reference Type',
 'Description 2',
 'Description',
 'Dimension Set ID',
 'Direct Unit Cost Price Factor',
 'Direct Unit Cost',
 'Expected Receipt Date',
 'Expected Receipt Time',
 'FA Posting Date',
 'Gen_ Bus_ Posting Group',
 'Gen_ Prod_ Posting Group',
 'Gross Weight',
 'Indirect Cost _',
 'Item Category Code',
 'Item Charge Base Amount',
 'Job Currency Factor',
 'Job Line Amount (LCY)',
 'Job Line Amount',
 'Job Line Disc_ Amount (LCY)',
 'Job Line Discount Amount',
 'Job Line

In [8]:
# drop columns that don't seem relevant to our challenge
to_drop=['Cross-Reference Type No_', 'Demand Query Line No_', 'Document No_', 'Item Rcpt_ Entry No_',
 'Line No_', 'Order Line No_', 'Pay-to Vendor No_', 'Pos_ No_', 'Prod_ Order Line No_', 'Routing Reference No_', 'Allow Invoice Disc_', 'Balance',
 'Bin Code', 'Contract Amount', 'Correction', 'Cross-Reference Type', 'Description 2', 'Dimension Set ID', 'Direct Unit Cost Price Factor',
 'Direct Unit Cost', 'Gen_ Bus_ Posting Group', 'Gen_ Prod_ Posting Group', 'Gross Weight', 'Indirect Cost _', 'Item Charge Base Amount',
 'Job Currency Factor', 'Job Line Amount (LCY)', 'Job Line Amount', 'Job Line Disc_ Amount (LCY)', 'Job Line Discount Amount', 'Job Line Discount _',
 'Job Total Price (LCY)', 'Job Unit Price (LCY)', 'Job Unit Price', 'Line Discount _', 'Net Weight', 'Overhead Rate', 'Pack Sample Quantity', 'Price Factor', 
 'Qty_ per Unit of Measure', 'Salvage Value', 'Shortcut Dimension 1 Code', 'Shortcut Dimension 2 Code','Qty_ Invoiced (Base)','Quantity (Base)',
 'Type', 'Unit Cost (LCY)', 'Unit Cost', 'Unit Price (LCY)', 'Unit Volume', 'Unit of Measure (Cross Ref_)', 'Unit of Measure Code',
 'Unit of Measure', 'Units per Parcel', 'VAT Base Amount', 'VAT Bus_ Posting Group', 'VAT Calculation Type', 'VAT Prod_ Posting Group','Blanket Order Line No_',
 'VAT _', 'Buy-from Vendor Name 2','Buy-from Address', 'timestamp','Item Category Code','Posting Group','Description','Buy-from Contact','Job Total Price','Quantity']
deliveries.drop(to_drop, inplace=True, axis=1)

In [9]:
# there are many columns with dates, let's analyse them with the goal of dropping the ones that are not needed
dates=deliveries[[
 'Expected Receipt Date', 'Expected Receipt Time', 'FA Posting Date', 'Lead Time Calculation',  'Planned Receipt Date',
 'Order Date', 'Original Date', 'Posting Date', 'Promised Receipt Date', 'Really Receipt DateTime', 'Requested Receipt Date']]
dates.head(3)

,Expected Receipt Date,Expected Receipt Time,FA Posting Date,Lead Time Calculation,Planned Receipt Date,Order Date,Original Date,Posting Date,Promised Receipt Date,Really Receipt DateTime,Requested Receipt Date
0,17.12.2019 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,NaN,17.12.2019 00:00:00,23.09.2019 00:00:00,17.12.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,16.12.2019 23:00:00,17.12.2019 00:00:00
1,17.12.2019 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,NaN,17.12.2019 00:00:00,15.11.2019 00:00:00,17.12.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,01.01.1753 00:00:00,17.12.2019 00:00:00
2,19.12.2019 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,NaN,19.12.2019 00:00:00,19.12.2019 00:00:00,19.12.2019 00:00:00,30.12.2019 00:00:00,19.12.2019 00:00:00,01.01.1753 00:00:00,19.12.2019 00:00:00


**Date columns to keep**

Order Date: date the order was created. 

Posting Date: date you want as the Posting Date on the Ledger Entry tables when the order is Shipped and Invoiced.

Promised Receipt Date: date your vendor promised they would deliver the goods. NAV does not change this date, so it allows you to set a marker to measure your vendor’s performance against what was promised.

Requested Receipt Date: date you wanted to receive the goods from the vendor. This date is not recalculated in NAV, so once it has been established, it gives you a marker to measure your vendor’s performance against the requested date.

Really Receipt DateTime

In [10]:
# searching for how Microsoft Nav calculates dates, we'll drop some columns indicating dates, taking into consideration the following:
# requested receipt date - lead time calculation = order date
# requested receipt date + inbound whse. handling time + safety lead time = expected receipt date
# lead time calculation: amount of time the product needs in order to be delivered to your company that the Purchasing Agent needs to be alerted
# Expected Receipt Date: date you currently expect your vendor to deliver the order
# Planned Receipt Date: date the order is expected to arrive at the warehouse location, uses the Requested Receipt Date
to_drop=['Expected Receipt Time','FA Posting Date','Lead Time Calculation','Planned Receipt Date','Original Date',]
deliveries.drop(to_drop, inplace=True, axis=1)
deliveries.head()


,Buy-from Vendor No_,No_,Order No_,Buy-from City,Buy-from Post Code,Buy-from Vendor Name,Expected Receipt Date,Location Code,Order Date,Order Quantity,Posting Date,Promised Receipt Date,Qty_ Rcd_ Not Invoiced,Quantity Invoiced,Really Receipt DateTime,Requested Receipt Date
0,K00421,00451185,EB19/2893,Altena-Dahle,58754,Möhling GmbH & Co. KG,17.12.2019 00:00:00,B-01,23.09.2019 00:00:00,"0,00000000000000000000",30.12.2019 00:00:00,17.12.2019 00:00:00,"0,00000000000000000000","22730,00000000000000000000",16.12.2019 23:00:00,17.12.2019 00:00:00
1,K02028,17000040,EB19/3532,Mörfelden-Walldorf,64546,RS Components GmbH,17.12.2019 00:00:00,B-01,15.11.2019 00:00:00,"0,00000000000000000000",30.12.2019 00:00:00,17.12.2019 00:00:00,"0,00000000000000000000","15,00000000000000000000",01.01.1753 00:00:00,17.12.2019 00:00:00
2,K04610,KS00520714,EB19/3570,Carei Jud. Satu Mare,RO - 445100,Taygan Metal Press SRL,19.12.2019 00:00:00,B-01,19.12.2019 00:00:00,"0,00000000000000000000",30.12.2019 00:00:00,19.12.2019 00:00:00,"0,00000000000000000000","2000,00000000000000000000",01.01.1753 00:00:00,19.12.2019 00:00:00
3,K04607,KS00525776-1,EB19/3737,Ebersbach,73061,Bodycote,06.01.2020 00:00:00,B-01,03.12.2019 00:00:00,"0,00000000000000000000",30.12.2019 00:00:00,06.01.2020 00:00:00,"0,00000000000000000000","300,00000000000000000000",01.01.1753 00:00:00,18.12.2019 00:00:00
4,K04137,MV00491619,EB19/2532,Velbert,42551,Heismann Drehtechnik,18.12.2019 00:00:00,B-01,03.07.2019 00:00:00,"0,00000000000000000000",30.12.2019 00:00:00,18.12.2019 00:00:00,"0,00000000000000000000","8704,00000000000000000000",01.01.1753 00:00:00,18.12.2019 00:00:00


Qty. Rcd. Not Invoiced Field: specifies how many units, such as pieces, of the ordered item have been received but not yet invoiced.

Quantity (Base): Inventory is stored in the base unit of measure, and every time that you sell or purchase or use any other unit of measure, NAV needs to know the quantity expressed in base unit of measure, so it can update inventory correctly

Order Quantity: covers all gross requirements (forecast, sales orders, and also replenishment of the inventory level).

In [11]:
deliveries.columns = deliveries.columns.str.replace(" ", "_").str.lower()
deliveries=deliveries.rename(columns={'buy-from_vendor_no_': 'vendor_no',"no_": "delivery_no","order_no_": "order_no",'buy-from_city':'city',
'buy-from_post_code':'postcode','buy-from_vendor_name':'vendor_name','expected_receipt_date':'expected_date','order_quantity':'order_qty',
 'promised_receipt_date': 'promised_date', 'qty__rcd__not_invoiced':'qty_rcd_not_invoiced','quantity_invoiced':'qty_invoiced','really_receipt_datetime':'delivery_date',
 'requested_receipt_date':'requested_date'})
deliveries.vendor_name=deliveries.vendor_name.str.replace(",", " ") # commas would create additional columns when exporting to csv
list(deliveries.columns)

['vendor_no',
 'delivery_no',
 'order_no',
 'city',
 'postcode',
 'vendor_name',
 'expected_date',
 'location_code',
 'order_date',
 'order_qty',
 'posting_date',
 'promised_date',
 'qty_rcd_not_invoiced',
 'qty_invoiced',
 'delivery_date',
 'requested_date']

In [12]:
# drop unnecessary characters
deliveries[['qty_invoiced','order_qty','qty_rcd_not_invoiced']] = deliveries[['qty_invoiced','order_qty','qty_rcd_not_invoiced']].applymap(lambda x: str(x)[:-21])
deliveries[['expected_date','order_date','posting_date','promised_date','delivery_date','requested_date']] = deliveries[['expected_date','order_date','posting_date','promised_date','delivery_date','requested_date']].applymap(lambda x: str(x)[:-9])

deliveries.head(3)

,vendor_no,delivery_no,order_no,city,postcode,vendor_name,expected_date,location_code,order_date,order_qty,posting_date,promised_date,qty_rcd_not_invoiced,qty_invoiced,delivery_date,requested_date
0,K00421,00451185,EB19/2893,Altena-Dahle,58754,Möhling GmbH & Co. KG,17.12.2019,B-01,23.09.2019,0,30.12.2019,17.12.2019,0,22730,16.12.2019,17.12.2019
1,K02028,17000040,EB19/3532,Mörfelden-Walldorf,64546,RS Components GmbH,17.12.2019,B-01,15.11.2019,0,30.12.2019,17.12.2019,0,15,01.01.1753,17.12.2019
2,K04610,KS00520714,EB19/3570,Carei Jud. Satu Mare,RO - 445100,Taygan Metal Press SRL,19.12.2019,B-01,19.12.2019,0,30.12.2019,19.12.2019,0,2000,01.01.1753,19.12.2019


In [ ]:
#deliveries = deliveries[(deliveries['expected_date']!= '01.01.1753')&(deliveries['expected_date']!= '09.09.2099')]
#deliveries=deliveries[~deliveries.promised_date.str.contains("2099")]
#deliveries=deliveries[~deliveries.delivery_date.str.contains("2099")]

In [13]:
# Some dates with a value of 01.01.1753 and 09.09.2099 mean the field is blank or there's been an error in the input. Thre are also dates with the year 2033.
# we will filter out these rows
deliveries=deliveries[deliveries.promised_date.str.contains("1753|2033|2099")==False]
deliveries=deliveries[deliveries.delivery_date.str.contains("1753|2033|2099")==False]
deliveries=deliveries[deliveries.expected_date.str.contains("1753|2033|2099")==False]
deliveries=deliveries[deliveries.order_date.str.contains("1753|2033|2099")==False]
deliveries=deliveries[deliveries.posting_date.str.contains("1753|2033|2099")==False]
deliveries=deliveries[deliveries.requested_date.str.contains("1753|2033|2099")==False]
deliveries.shape

(4152, 16)

In [14]:
# typecast
deliveries = deliveries.astype({'order_date': 'datetime64','posting_date': 'datetime64','promised_date': 'datetime64',
'delivery_date': 'datetime64','requested_date': 'datetime64','expected_date':'datetime64','qty_invoiced': 'int64','order_qty': 'int64','qty_rcd_not_invoiced': 'int64'})
deliveries.dtypes

vendor_no                       object
delivery_no                     object
order_no                        object
city                            object
postcode                        object
vendor_name                     object
expected_date           datetime64[ns]
location_code                   object
order_date              datetime64[ns]
order_qty                        int64
posting_date            datetime64[ns]
promised_date           datetime64[ns]
qty_rcd_not_invoiced             int64
qty_invoiced                     int64
delivery_date           datetime64[ns]
requested_date          datetime64[ns]
dtype: object

In [15]:
# we will drop rows that have a delivery, promised or order date later than 01.03.2022
# also, drop rows where order date is later than delivery, expected, requested or posting dates. we assume this is an input error.
deliveries = deliveries[(deliveries['promised_date']< '2022-03-01')&(deliveries['delivery_date']< '2022-03-01')&(deliveries['order_date']< '2022-03-01')]
deliveries = deliveries[(deliveries['promised_date']> deliveries['order_date'])&(deliveries['delivery_date']> deliveries['order_date'])&(deliveries['expected_date']> deliveries['order_date'])&(deliveries['requested_date']> deliveries['order_date'])&(deliveries['posting_date']> deliveries['order_date'])]

deliveries.shape

(2334, 16)

In [16]:
# drop rows with nan values
deliveries.isna().sum()
deliveries = deliveries.dropna()
deliveries.shape

(2332, 16)

In [17]:
# drop duplicates
deliveries.drop_duplicates(keep='first')
deliveries.shape

(2332, 16)

In [18]:
# for our analysis, we will upload world cities database from: https://simplemaps.com/data/world-cities to insert country column
cities = pd.read_csv(r'worldcities.csv')
cities=cities[["city", "country", "iso2"]]
cities.head()

,city,country,iso2
0,Tokyo,Japan,JP
1,Jakarta,Indonesia,ID
2,Delhi,India,IN
3,Mumbai,India,IN
4,Manila,Philippines,PH


In [19]:
# filter only the cities that are also present in the deliveries dataframe
city_list=list(set(deliveries.city))
city_list
cities=cities[cities.city.isin(city_list)].reset_index(drop=True)
cities

,city,country,iso2
0,Tokyo,Japan,JP
1,Berlin,Germany,DE
2,Hamburg,Germany,DE
3,Frankfurt,Germany,DE
4,Düsseldorf,Germany,DE
...,...,...,...
159,Wirges,Germany,DE
160,Hirschau,Germany,DE
161,Berlin,United States,US
162,Wilhelmsdorf,Germany,DE


In [20]:
# check for possible duplication of city names in the world cities dataframe
duplicate = cities[cities.duplicated(subset="city", keep=False)].drop_duplicates(keep='first').reset_index(drop=True).sort_values(by='city') # False marks all duplicates as True
duplicate

# as we can see, some city names are present in more than one country. If we were to merge the two dataframs, we could have rows assigned to the wrong country
# But as we know that the dataset is from a German company, we could infer that most of these vendors are in Germany.
# In order to confirm this, we will match a list of German postal codes to the postal codes in the deliveries dataframe

,city,country,iso2
0,Berlin,Germany,DE
9,Berlin,United States,US
13,Birkenfeld,Germany,DE
16,Feldkirchen,Germany,DE
11,Feldkirchen,Austria,AT
3,Halle,Germany,DE
8,Halle,Belgium,BE
5,Hamburg,United States,US
1,Hamburg,Germany,DE
18,Herne,United Kingdom,GB


In [21]:
# we will first match zip code to country by uploading German postcode list. Source: https://www.suche-postleitzahl.org/downloads
de_postcodes = pd.read_excel(r'zuordnung_plz_ort.xlsx')
de_postcodes=de_postcodes.rename(columns={'ort': 'city',"plz": "postcode"})

# we're only interested in the postcode and city columns. we add a country column with Germany as a value
de_postcodes=de_postcodes[["city","postcode"]]
de_postcodes['country']='Germany'
de_postcodes = pd.DataFrame(data=de_postcodes)
de_postcodes.postcode = de_postcodes.postcode.astype(str)
de_postcodes.head()


,city,postcode,country
0,Aach,78267,Germany
1,Aach,54298,Germany
2,Aachen,52062,Germany
3,Aachen,52064,Germany
4,Aachen,52066,Germany


In [202]:
# we merge deliveries and de_postcode dataframes on postcode
# drop duplicated column city_y and copy country column
joined_df=pd.merge(deliveries,de_postcodes, on="postcode", how="left").drop(columns="city_y").rename(columns={"city_x": "city", "country":"country2"})
joined_df.insert(4, 'country',joined_df["country2"])
joined_df=joined_df.drop(columns="country2")
joined_df.head()

,vendor_no,delivery_no,order_no,city,country,postcode,vendor_name,expected_date,location_code,order_date,order_qty,posting_date,promised_date,qty_rcd_not_invoiced,qty_invoiced,delivery_date,requested_date
0,K00421,00451185,EB19/2893,Altena-Dahle,NaN,58754,Möhling GmbH & Co. KG,2019-12-17,B-01,2019-09-23,0,2019-12-30,2019-12-17,0,22730,2019-12-16,2019-12-17
1,K00016,40063527,EB19/3307,Berlin,Germany,13437,Höttges,2019-12-13,A-01,2019-10-29,0,2019-12-30,2019-12-13,0,3200,2019-12-18,2019-12-13
2,K00016,40063526,EB19/3307,Berlin,Germany,13437,Höttges,2019-12-13,A-01,2019-10-29,0,2019-12-30,2019-12-13,0,3200,2019-12-16,2019-12-13
3,K00268,27602381,EB19/2763,Salzweg,Germany,94121,Electrovac Hacht & Huber GmbH,2019-12-13,A-01,2019-08-22,0,2019-12-30,2019-12-13,0,1014,2019-12-19,2019-12-13
4,K03974,HES0076418,EB19/2849,Cisano Bergamosco,NaN,IT-24034,M.S. Ambrogio S.p.A.,2020-10-01,A-01,2019-03-09,0,2019-12-30,2020-10-01,0,54780,2019-12-17,2020-10-01


In [203]:
# let's verify the rows with NaN values in the country column and try to fill them up with the cities dataframe
is_NaN = joined_df[["city", "country"]].isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = joined_df[row_has_NaN]
rows_with_NaN=rows_with_NaN[["city", "country"]].reset_index(drop=True)
rows_with_NaN

# create new_countries dataframe with the rows where a country was assigned
new_countries=pd.merge(rows_with_NaN,cities, on="city", how="left").drop(columns="country_x").rename(columns={"country_y":"country"}).dropna().drop_duplicates(keep='first').reset_index(drop=True)
new_countries


,city,country,iso2
0,Blaichach,Germany,DE
1,Chemnitz,Germany,DE
2,Wildeshausen,Germany,DE
3,Harelbeke,Belgium,BE
4,Bydgoszcz,Poland,PL
5,Lüdenscheid,Germany,DE
6,Mannheim,Germany,DE
7,Velbert,Germany,DE
8,Tokyo,Japan,JP
9,Iserlohn,Germany,DE


In [204]:
#check for null values
joined_df.country.isnull().sum()

284

In [205]:
# merge new_countries with joined_df and fill up empty cells in country column
joined_df=pd.merge(joined_df, new_countries, on="city", how="left")
joined_df.loc[joined_df['country_x'].isnull(),'country_x'] = joined_df['country_y'] # replace NaN values with values from a column of another dataframe
joined_df=joined_df.rename(columns={"country_x":"country"}).drop(columns=["country_y"])
joined_df.head()

#check for remaining null values
joined_df.country.isnull().sum()
# almost one hundred rows were matched

193

In [206]:
# let's verify again for null values in country
is_NaN = joined_df[["city", "country", "postcode"]].isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = joined_df[row_has_NaN]
rows_with_NaN=rows_with_NaN[["city", "country", "postcode", "iso2"]].drop_duplicates(keep='first').reset_index(drop=True)
rows_with_NaN
# as we can see, there are some inconsistencies in the city column: combination of city name a country name or county name, wrong use of capital letters, typos, among others.
# since some postcodes include the ISO2 country code, making it easier to identify the country, we will first tackle this.

,city,country,postcode,iso2
0,Altena-Dahle,NaN,58754,NaN
1,Cisano Bergamosco,NaN,IT-24034,NaN
2,Großpostwitz,NaN,02692,NaN
3,Reichshof - Eckenhagen,NaN,51574,NaN
4,Offenbach,NaN,63019,NaN
5,Zhejiang (China),NaN,CN-311201,NaN
6,Bad Malente,NaN,23711,NaN
7,Markranstaedt-Fran,NaN,04420,NaN
8,Pozuelo de Alarcón (Madrid),NaN,ES-28224,NaN
9,QUINCY VOISINS,NaN,77860,NaN


In [207]:
# extract iso2 country code from postcode column
rows_with_NaN.iso2=rows_with_NaN.postcode.str.split('-').str[0]

rows_with_NaN['country'] = rows_with_NaN['iso2'].apply(lambda value: 'Italy' if value=='IT' 
else 'China' if value == 'CN'
else 'Spain' if value == 'ES' 
else 'Romania' if value == 'RO' 
else 'Netherlands' if value == 'NL'
else 'France' if value == 'FR'
else float('NaN')) # to fill it with NaN values

rows_with_NaN.iloc[19]=rows_with_NaN.iloc[19].fillna("France")

#rows_with_NaN.country.iloc[0]
rows_with_NaN

,city,country,postcode,iso2
0,Altena-Dahle,NaN,58754,58754
1,Cisano Bergamosco,Italy,IT-24034,IT
2,Großpostwitz,NaN,02692,02692
3,Reichshof - Eckenhagen,NaN,51574,51574
4,Offenbach,NaN,63019,63019
5,Zhejiang (China),China,CN-311201,CN
6,Bad Malente,NaN,23711,23711
7,Markranstaedt-Fran,NaN,04420,04420
8,Pozuelo de Alarcón (Madrid),Spain,ES-28224,ES
9,QUINCY VOISINS,NaN,77860,77860


In [208]:
# replace NaN values with values from a column of rows_with_NaN dataframe
rows_with_NaN=pd.merge(rows_with_NaN, de_postcodes, on="city", how="left")
rows_with_NaN.loc[rows_with_NaN['country_x'].isnull(),'country_x'] = rows_with_NaN['country_y']
rows_with_NaN=rows_with_NaN.drop(columns=["postcode_y","country_y", "postcode_x", "iso2"]).drop_duplicates(keep='first').reset_index(drop=True)
rows_with_NaN

new_countries=rows_with_NaN.rename(columns={"country_x": "country"}).dropna().drop_duplicates(keep='first').reset_index(drop=True)
new_countries

,city,country
0,Cisano Bergamosco,Italy
1,Zhejiang (China),China
2,Pozuelo de Alarcón (Madrid),Spain
3,Tione di Trento,Italy
4,Carei Jud. Satu Mare,Romania
5,Saint Claude,France
6,Habsheim,France
7,Kraftsdorf,Germany
8,El Son,Netherlands
9,Bellignat,France


In [209]:
# merge new_countries with joined_df and fill up empty cells in country column
joined_df=pd.merge(joined_df, new_countries, on="city", how="left")
joined_df.loc[joined_df['country_x'].isnull(),'country_x'] = joined_df['country_y'] # replace NaN values with values from a column of another dataframe
joined_df=joined_df.rename(columns={"country_x":"country"}).drop(columns=["country_y", "iso2"])
joined_df.city = joined_df.city.str.title()

joined_df.head()

,vendor_no,delivery_no,order_no,city,country,postcode,vendor_name,expected_date,location_code,order_date,order_qty,posting_date,promised_date,qty_rcd_not_invoiced,qty_invoiced,delivery_date,requested_date
0,K00421,00451185,EB19/2893,Altena-Dahle,NaN,58754,Möhling GmbH & Co. KG,2019-12-17,B-01,2019-09-23,0,2019-12-30,2019-12-17,0,22730,2019-12-16,2019-12-17
1,K00016,40063527,EB19/3307,Berlin,Germany,13437,Höttges,2019-12-13,A-01,2019-10-29,0,2019-12-30,2019-12-13,0,3200,2019-12-18,2019-12-13
2,K00016,40063526,EB19/3307,Berlin,Germany,13437,Höttges,2019-12-13,A-01,2019-10-29,0,2019-12-30,2019-12-13,0,3200,2019-12-16,2019-12-13
3,K00268,27602381,EB19/2763,Salzweg,Germany,94121,Electrovac Hacht & Huber GmbH,2019-12-13,A-01,2019-08-22,0,2019-12-30,2019-12-13,0,1014,2019-12-19,2019-12-13
4,K03974,HES0076418,EB19/2849,Cisano Bergamosco,Italy,IT-24034,M.S. Ambrogio S.p.A.,2020-10-01,A-01,2019-03-09,0,2019-12-30,2020-10-01,0,54780,2019-12-17,2020-10-01


In [210]:
joined_df.country.isnull().sum()

135

In [211]:
joined_df.head()

,vendor_no,delivery_no,order_no,city,country,postcode,vendor_name,expected_date,location_code,order_date,order_qty,posting_date,promised_date,qty_rcd_not_invoiced,qty_invoiced,delivery_date,requested_date
0,K00421,00451185,EB19/2893,Altena-Dahle,NaN,58754,Möhling GmbH & Co. KG,2019-12-17,B-01,2019-09-23,0,2019-12-30,2019-12-17,0,22730,2019-12-16,2019-12-17
1,K00016,40063527,EB19/3307,Berlin,Germany,13437,Höttges,2019-12-13,A-01,2019-10-29,0,2019-12-30,2019-12-13,0,3200,2019-12-18,2019-12-13
2,K00016,40063526,EB19/3307,Berlin,Germany,13437,Höttges,2019-12-13,A-01,2019-10-29,0,2019-12-30,2019-12-13,0,3200,2019-12-16,2019-12-13
3,K00268,27602381,EB19/2763,Salzweg,Germany,94121,Electrovac Hacht & Huber GmbH,2019-12-13,A-01,2019-08-22,0,2019-12-30,2019-12-13,0,1014,2019-12-19,2019-12-13
4,K03974,HES0076418,EB19/2849,Cisano Bergamosco,Italy,IT-24034,M.S. Ambrogio S.p.A.,2020-10-01,A-01,2019-03-09,0,2019-12-30,2020-10-01,0,54780,2019-12-17,2020-10-01


In [212]:
is_NaN = joined_df[["city", "country", "postcode"]].isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = joined_df[row_has_NaN]
rows_with_NaN=rows_with_NaN[["city", "country", "postcode"]].drop_duplicates(keep='first').reset_index(drop=True)
rows_with_NaN

,city,country,postcode
0,Altena-Dahle,NaN,58754
1,Großpostwitz,NaN,02692
2,Reichshof - Eckenhagen,NaN,51574
3,Offenbach,NaN,63019
4,Bad Malente,NaN,23711
5,Markranstaedt-Fran,NaN,04420
6,Quincy Voisins,NaN,77860
7,Schonach,NaN,78133
8,Luedenscheid,NaN,58486


In [ ]:
# export as csv file
deliveries.to_csv(r'deliveries_clean.csv', index=False);

In [ ]:
# export to Excel file
read_file = pd.read_csv (r'deliveries_clean.csv')
read_file.to_excel (r'deliveries.xlsx', index = None, header=True)
